In [1]:
#Import of Useful Libraries

import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  # Import the numpy library
import random as rdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.interpolate import griddata


In [2]:
#I define a function for rotation of the stress tensor into geographical coordinates

def Geo_rot(Alpha, Betha, Gamma, STens):
    
    Alpha = Alpha*2*math.pi/360
    Betha = Betha*2*math.pi/360
    Gamma = Gamma*2*math.pi/360
    
    RPG = np.array([[np.cos(Alpha)*np.cos(Betha),                                  np.sin(Alpha)*np.cos(Betha),                                    -np.sin(Betha)], 
           [np.cos(Alpha)*np.sin(Betha)*np.sin(Gamma)-np.sin(Alpha)*np.cos(Gamma), np.sin(Alpha)*np.sin(Betha)*np.sin(Gamma)+np.cos(Alpha)*np.cos(Gamma) , np.cos(Betha)*np.sin(Gamma)],
           [np.cos(Alpha)*np.sin(Betha)*np.cos(Gamma)+np.sin(Alpha)*np.sin(Gamma), np.sin(Alpha)*np.sin(Betha)*np.cos(Gamma)-np.cos(Alpha)*np.sin(Gamma) , np.cos(Betha)*np.cos(Gamma)]])

    # Transpose RPG

    RPG_T = RPG.transpose()

    # Calculate Stress Tensor in Geographical Coordinate System
    # SGEO = RGP_T * STens_E * RGP

    SGEO = np.round(np.dot(RPG_T,np.dot(STens,RPG)))


    return SGEO

In [3]:
#I define useful conversions of nomenclature for sin and cosine

def sin(x) :
    a = np.sin(x)
    return a

def cos(x) :
    a = np.cos(x)
    return a


In [4]:
#I define a function for rotation of the stress tensor in geographical coordinates to wellbore coordinates

def Well_rot(delta, phi, STens): #phi is the angle with respect to the vertical axis, delta is the azimuth

    delta = delta*2*math.pi/360
    phi = phi*2*math.pi/360
        
      
    RB = np.array([[-np.cos(delta)*np.cos(phi),           -np.sin(delta)*np.cos(phi),          np.sin(phi)], 
           [np.sin(delta),                                         -np.cos(delta) ,                 0],
           [np.cos(delta)*np.sin(phi)   ,                          np.sin(delta)*np.sin(phi)    ,       np.cos(phi)]])

    # Transpose RB

    RB_T = RB.transpose()

    # Calculate Stress Tensor in Wellbore Coordinate System
    # SWell = RB * STens * RB_T

    SWell = np.round(np.dot(RB,np.dot(STens,RB_T)),2)


    return SWell

In [5]:
#I define a function to calculate the stresses around the wellbore wall based on Kirsch equations
# The output of this function are the max and min principal stresses for an angle theta in the wellbore wall
# The principal stresses come from the eigen values, in which the remaining effective stress = sigma_rr


In [6]:
def kirsch(tens, theta, PW, PP, Poisson):
    
    theta = theta*2*math.pi/360    
      
    sigma11 = tens[0][0] - PP
    sigma12 = tens[0][1]
    sigma13 = tens[0][2]
    sigma22 = tens[1][1] - PP
    sigma23 = tens[1][2]
    sigma33 = tens[2][2] - PP
    
    sigma_zz =  sigma33 - 2*Poisson*(sigma11 - sigma22)*np.cos(2*theta) - 4*Poisson*sigma12*np.sin(2*theta)
    sigma_tt = sigma11 + sigma22 - 2*(sigma11 - sigma22)*np.cos(2*theta) - 4*sigma12*np.sin(2*theta) - (PW - PP)
    tau_tz = 2*(sigma23*np.cos(theta) - sigma13 * np.sin(theta))
    sigma_rr = PW - PP
    
    sigma_max = round( (sigma_zz + sigma_tt) / 2 + math.sqrt(((sigma_zz-sigma_tt)/2)**2 + tau_tz**2) , 3)
    sigma_min = round( (sigma_zz + sigma_tt) / 2 - math.sqrt(((sigma_zz-sigma_tt)/2)**2 + tau_tz**2) , 3)

    return sigma_max, sigma_min

In [7]:
# I define the function 'Tensile': it loops through all possible combinations of wellbore deviation and azimuth
# in order to calculate the tensile Fractures Likelihood and the Breakout Likelihood (the name is probably not the best...)

def Tensile(Alpha, Beta, Gamma, S1, S2, S3, PW, PP, Poisson, inc = 0, azi = 0):
    
    s_tensor = np.array([[S1, 0 , 0], [0, S2, 0], [0, 0, S3]] )

    geo_s = Geo_rot(Alpha, Beta, Gamma, s_tensor) #principal stresses to geographical coordinates

    phi = np.linspace(0, 90, 90) #well dip array
    delta = np.linspace(0,180, 180) #well azimuth array
    angle = np.linspace(0,180, 180) #well border angle

    UCS = [] #UCS array
    T0 = []  # Tensile strength array
    Mohr_w = [] # width by Mohr criterium
    DP_w = [] # width by Drucker Prager criterium
    ML_w = [] #width by Modified Lade criterium
    x = [] #x coordinate for plotting
    y = [] #y coordinate for plotting
    deltar = []
    phir = []
    
    for p in phi:
        for d in delta:

            x.append(p*math.sin(d*2*math.pi/360)) #cos and sin are inverted because of the plot
            y.append(p*math.cos(d*2*math.pi/360))
            deltar.append(d*2*math.pi/360)
            phir.append(p)
            
            well_s = Well_rot(d, p, geo_s) #principal stress tensor to well coordinates
            max_s_list = []
            min_s_list = []
            width_MC = 0
            width_DP = 0
            width_ML = 0

            for ang in angle:
                max_s, min_s = kirsch(well_s, ang, PW, PP, Poisson) #calculate min and max stress for each point in the well
                max_s_list.append(max_s)
                min_s_list.append(min_s)

                #max_s, min_s = kirsch(well_s, ang, PW_2, PP, Poisson) #calculate min and max stress again w/ a different PW

                #min_k = min(min_s, max_s, (PW_2 - PP))

                #I1 = max_s + (PW_2-PP) + min_s 
                #sqJ2 = math.sqrt(1/6*( (max_s - (PW_2-PP))**2 + (min_s - (PW_2 - PP))**2 + (max_s - min_s)**2))
                #ML_I1 = (max_s + ML_S) + ((PW_2-PP) + ML_S) + (min_s + ML_S)
                #ML_I3 = (max_s + ML_S) * ((PW_2-PP) + ML_S) * (min_s + ML_S)

                #Mohr_sigma1 = min_k*q_Q1 + UCS_Q1 #Mohr Coulomb

                #DP_sqJ2 = C3_Q1 + C4_Q1*I1 #Drucker Prager

                #ML_I1_f = ((27 + ML_n)*ML_I3)**(1/3) #Modified Lade

                #if max_s > Mohr_sigma1: #condition for failure following Mohr_Coulomb criterium
                #    width_MC += max(angle)/len(angle)

                #if sqJ2 > DP_sqJ2 : #condition for failure following Drucker-Prager criterium
                #    width_DP += max(angle)/len(angle)

                #if ML_I1 > ML_I1_f : #condition for failure following Modified-Lade criterium
                #    width_ML += max(angle)/len(angle)

            UCS_n = max(max_s_list)
            T0_n = min(min_s_list) 

            UCS.append(UCS_n)
            T0.append(T0_n)
            Mohr_w.append(width_MC/2)
            DP_w.append(width_DP/2)
            ML_w.append(width_ML/2)

    # Duplicating the different arrays to get the results for the full circle
    UCS += UCS.copy()
    T0 += T0.copy()
    Mohr_w += Mohr_w.copy()
    DP_w += DP_w.copy()
    ML_w += ML_w.copy()

    x= np.array(x)
    x_m = x*(-1)
    x = np.append(x, x_m )

    y= np.array(y)
    y_m = y*(-1)
    y = np.append(y, y_m )

    deltar = np.array(deltar)
    deltar_m = deltar+np.pi
    deltar = np.append(deltar, deltar_m )

    phir = np.array(phir)
    phir_m = phir
    phir = np.append(phir, phir_m )

    # Mesh definition for imshow plot - this was not used in the end
    
 #   points = np.stack([deltar, phir], axis=1)
 #   x = np.linspace(np.min(deltar), np.max(deltar), 101)
 #   y = np.linspace(np.min(phir), np.max(phir), 101)
 #   grid_x, grid_y = np.meshgrid(deltar, phir) 
 #   grid_T0 = griddata(points, T0, (grid_x, grid_y), method='linear')
 #   grid_UCS = griddata(points, UCS, (grid_x, grid_y), method='linear')

    #Tendency for tensile failure considering a constant Ts criterium

    fig, ax = plt.subplots(ncols = 2, subplot_kw={'projection': 'polar'}, figsize = (10, 8))
    fig.tight_layout(pad=3.0)
    
    color_map = plt.cm.get_cmap('jet')
    reversed_color_map = color_map.reversed()
    plot_T0 = ax[0].scatter(deltar, phir, c=T0, cmap=reversed_color_map )
#    plot_T0 = ax.imshow(grid_T0, extent=(np.min(deltar),np.min(deltar),np.min(phir),np.max(phir)), origin='lower')
    cb_0 = plt.colorbar(plot_T0, ax=ax[0] , orientation = 'horizontal')
    ax[0].set_rmax(90)
    ax[0].set_rticks([0, 30, 60, 90])  # Less radial ticks
  #  ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
    ax[0].set_theta_zero_location('N')
    ax[0].set_theta_direction(-1)
    ax[0].grid(True)
    cb_0.set_label('Ts (psi)', fontsize=11)

    #Tendency for breakouts considering a constant UCS criterium 
    
    plot_UCS = ax[1].scatter(deltar, phir, c=UCS , cmap='jet')
#    plot_UCS = ax.imshow(grid_UCS, extent=(np.min(deltar),np.min(deltar),np.min(phir),np.max(phir)), origin='lower')
    cb_1 = plt.colorbar(plot_UCS, ax = ax[1], orientation = 'horizontal')
    ax[1].set_rmax(90)
    ax[1].set_rticks([0, 30, 60, 90])  # Less radial ticks
  #  ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
    ax[1].set_theta_zero_location('N')
    ax[1].set_theta_direction(-1)
    ax[1].grid(True)
    cb_1.set_label('UCS (psi)', fontsize=11)
    
    if inc > 0:
        ax[0].scatter(azi, inc, c='violet', s=30,  edgecolors='black')
        ax[0].annotate('Well', (azi-math.pi/180*10, inc+5), color = 'violet', size=12, weight='bold')
        ax[1].scatter(azi, inc, c='violet', s=30, edgecolors='black')
        ax[1].annotate('Well', (azi-math.pi/180*10, inc+5), color='violet', size=12, weight='bold')

In [8]:
#Test to try the code (it worked!)

#Normal Regime

#Alpha =160
#Beta = 90
#Gamma = 0

#Stresses = [1000, 1500, 2000]

#S1 = max(Stresses)
#S3 = min(Stresses)
#S2 = Stresses[Stresses not in [S1, S3]]
#PP = 800 #psi
#PW = 800 #psi mud pressure
#Poisson = 0.3

#Tensile(Alpha, Beta, Gamma, S1, S2, S3, PW, PP, Poisson, 1, 5.5)  


In [9]:
#Defining Mohr circle function - This was originally intended to be used, but in the end it exceeded the scope of the project

def Mohr(St1, St2, St3):

    
    #Mohr Definition
    
  
    Center1 = (St3 + St2)/2
    Center2 = (St2 + St1)/2
    Center3 = (St3 + St1)/2

    Radius1 = (St2 - St3)/2
    Radius2 = (St1 - St2)/2
    Radius3 = (St1 - St3)/2


    
    theta_end = math.pi
    theta_step = math.pi/360
    theta = np.arange(0,theta_end+theta_step,theta_step)

    #Define the arrays
    X1_arr = np.arange(0,361)
    Y1_arr = np.arange(0,361)
    X2_arr = np.arange(0,361)
    Y2_arr = np.arange(0,361)
    X3_arr = np.arange(0,361)
    Y3_arr = np.arange(0,361)


    X1_arr = Radius1 * np.cos(theta) + Center1
    Y1_arr = Radius1 * np.sin(theta)

    X2_arr = Radius2 * np.cos(theta) + Center2
    Y2_arr = Radius2 * np.sin(theta)

    X3_arr = Radius3 * np.cos(theta) + Center3
    Y3_arr = Radius3 * np.sin(theta)

   # fig, ax  = plt.subplots()
    
    #f = plt.figure()
    #f.set_figwidth(10)
    #f.set_figheight(10)
  
  
    plt.plot(X1_arr, Y1_arr, X2_arr, Y2_arr, X3_arr, Y3_arr)


    # setting x and y axis range
    plt.ylim(0,2000)
    plt.xlim(0,4000)

    # naming the x axis
    plt.xlabel('Normal Stress [MPa]')
    # naming the y axis
    plt.ylabel('Shear Stress [MPa]')

    # naming the plot    
    plt.title('Stress Tensor')
    plt.axis('square')
    